# 🌐 Real-Time RAG Web Frontend via Blazor + Backend via Jupyter C#

This notebook provides code and architecture guidance for integrating:
- **Backend in .NET Interactive (C#)** using OpenAI + Lucene.NET
- **Frontend in Blazor WebAssembly or Server** for real-time user interaction

This hybrid RAG pipeline returns responses based on user queries using OpenAI embeddings + BM25.

✅ You will build:
- A Web API backend (Blazor Server)
- An embedded RAG inference endpoint
- Frontend UI to submit questions and render answers

In [ ]:
// Backend API Controller for RAG Query
using Microsoft.AspNetCore.Mvc;
[ApiController]
[Route("api/[controller]")]
public class RagController : ControllerBase
{
    [HttpPost("query")]
    public async Task<IActionResult> Query([FromBody] QueryRequest request)
    {
        var chunks = await HybridSearch(request.Question);
        var prompt = $"Answer the following using context:\n{string.Join("\n---\n", chunks)}\n\nQ: {request.Question}\nA:";
        var response = await CallOpenAI(prompt);
        return Ok(new { answer = response });
    }
    public record QueryRequest(string Question);
}

## 🖥️ Blazor Web UI (Pages/Rag.razor)

In [ ]:
@page "/rag"
<h3>Ask RAG AI</h3>

<input @bind="question" placeholder="Ask a question" class="form-control" />
<button class="btn btn-primary mt-2" @onclick="SendQuery">Ask</button>

@if (answer != null)
{
    <div class="alert alert-info mt-3">
        <b>Answer:</b> @answer
    </div>
}

@code {
    string question;
    string answer;

    async Task SendQuery()
    {
        var request = new { question = question };
        var http = new HttpClient();
        var resp = await http.PostAsJsonAsync("/api/rag/query", request);
        var result = await resp.Content.ReadFromJsonAsync<JsonElement>();
        answer = result.GetProperty("answer").GetString();
    }
}

## 🔧 Integration Notes
- Run the RAG pipeline service in .NET (with `Program.cs` for Web API hosting)
- OpenAI key via `IConfiguration`
- You can expand to stream token-by-token using OpenAI chat streaming and SignalR

To deploy:
1. `dotnet new blazorserver -o RAGWeb`
2. Add API controller and Razor page above
3. Inject backend logic via service interface or shared project
4. Run with `dotnet run`

**Optionally**, host API separately and call it from **Streamlit** or JS frontend with `fetch()`